In [ ]:
# default_exp nanopore

# nanopore

In [ ]:
# export

import pandas as pd
import pysam
import scipy.stats as stats

from simplebitk.utils.multirun import parallel_run

In [ ]:
# export

def polya_in_nanopolish(polya):
    df = pd.read_csv(polya,sep='\t')
    return df[df['qc_tag']=='PASS']


In [ ]:
# export

def polya_length_in_gene(bam,contig,polya):
    with pysam.AlignmentFile(bam,'rb') as buf:
        polya_df = polya_in_nanopolish(polya)
        contig_b = buf.fetch(contig=contig,multiple_iterators=True)
        query_names= []
        for i in contig_b:
            query_names.append(i.query_name)
        if len(query_names) > 0:
            return contig,polya_df[polya_df['readname'].isin(query_names)]['polya_length'].median()
        else:
            return contig,None
    

In [ ]:
# export

def polya_length_correlation(transcriptome_bam,polya,nanocount,outdir,*,contigs=None,thread=10):
    '''
    
    :param str transcriptome_bam: alignments of reads to transcriptome
    :param str polya: polya results from nanopolish polya
    :param str nanocount: nanocount result
    :param str outdir: outdir
    :param list[str] contigs: contigs
    :param int thread: thread
    '''
    with pysam.AlignmentFile(transcriptome_bam,'rb') as buf:
        if None is contigs:
            genes = buf.references
        else:
            genes = contigs
    
    paras = [[transcriptome_bam,gene, polya] for gene in genes]
    results = parallel_run(polya_length_in_gene,paras, thread=thread)
    p_df=pd.DataFrame(results,columns=['transcript','polya_length'])
    tpm = pd.read_csv(nanocount,sep='\t')
    m_df=tpm.merge(p_df,how='left',left_on='transcript_name',right_on='transcript',)
    cor_df = m_df.dropna()
    cor,p = stats.pearsonr(cor_df['tpm'],cor_df['polya_length'])
    print(f'Cor:{cor},and P value: {p}')
    m_df.to_csv(f'{outdir}/polya_length_tpm.csv',index=False)

In [ ]:
bam='tests/transcriptome_sorted.bam'
polya_nanopolish='tests/polya.txt'
tpm='tests/transcriptome.tsv'
gene_list=['PNW87518',
 'PNW80207',
 'PNW77299',
 'PNW87372',
 'PNW74640',
 'PNW81711',
 'PNW72105',
 'PNW75489',
 'PNW80672',
 'PNW71721',
 'PNW77897',
 'PNW75504',
 'PNW82095',
 'PNW79633',
 'PNW82527',
 'PNW75662',
 'PNW88381',
 'PNW76429',
 'PNW75062',
 'PNW73075',
 'PNW77066',
 'PNW73697',
 'PNW77976',
 'PNW85836',
 'PNW73894',
 'PNW79546',
 'PNW79547',
 'PNW78248',
 'PNW76141',
 'PNW79605',
 'PNW85749',
 'PNW82080',
 'PNW74495',
 'PNW77005',
 'PNW80843',
 'PNW75583',
 'PNW85356',
 'PNW75710',
 'PNW73294',
 'PNW86545',
 'PNW89085',
 'PNW79146',
 'PNW79147',
 'PNW87045',
 'PNW78353',
 'PNW87921',
 'PNW88677',
 'PNW83197',
 'PNW83138',
 'PNW75143',
 'PNW76070',
 'PNW81500',
 'PNW88855',
 'PNW76927',
 'PNW86940',
 'PNW83198',
 'PNW75283',
 'PNW73691',
 'PNW83629',
 'PNW73166',
 'PNW77800',
 'PNW74925',
 'PNW75512',
 'PNW69981',
 'PNW78816',
 'PNW78817',
 'PNW76085',
 'PNW80960',
 'PNW74228',
 'PNW78655',
 'PNW81274',
 'PNW81805',
 'PNW82533',
 'PNW70544',
 'PNW84358',
 'PNW86172',
 'PNW85627',
 'PNW71968',
 'PNW79354',
 'PNW74304',
 'PNW88737',
 'PNW74010',
 'PNW82575',
 'PNW76422',
 'PNW84918',
 'PNW76019',
 'PNW86341',
 'PNW87371',
 'PNW83660',
 'PNW80902',
 'PNW79327',
 'PNW79326',
 'PNW80804',
 'PNW72305',
 'PNW70449',
 'PNW73756',
 'PNW71338',
 'PNW85419',
 'PNW84943',
 'PNW83328',
 'PNW88792',
 'PNW78970',
 'PNW82073',
 'PNW87056',
 'PNW87345',
 'PNW73969',
 'PNW87012',
 'PNW85291',
 'PNW81378',
 'PNW82591',
 'PNW75765',
 'PNW78887',
 'PNW83288',
 'PNW77523',
 'PNW73742',
 'PNW81860',
 'PNW73415',
 'PNW69991',
 'PNW85924',
 'PNW84701',
 'PNW76650',
 'PNW70527',
 'PNW86194',
 'PNW77396',
 'PNW70032',
 'PNW77053',
 'PNW70031',
 'PNW84610',
 'PNW82324',
 'PNW82333',
 'PNW74486',
 'PNW72940',
 'PNW74569',
 'PNW86286',
 'PNW73110',
 'PNW84917',
 'PNW84386',
 'PNW82001',
 'PNW72608',
 'PNW83946',
 'PNW86377',
 'PNW79249',
 'PNW74433',
 'PNW73803',
 'PNW86185',
 'PNW71657',
 'PNW77185',
 'PNW77155',
 'PNW70002',
 'PNW87240',
 'PNW70605',
 'PNW81721',
 'PNW81778',
 'PNW76005',
 'PNW81426',
 'PNW86547',
 'PNW75922',
 'PNW73506',
 'PNW79922',
 'PNW77697',
 'PNW88945',
 'PNW87134',
 'PNW76412',
 'PNW75778',
 'PNW82906',
 'PNW82805',
 'PNW77810',
 'PNW86758',
 'PNW89084',
 'PNW76373',
 'PNW77517',
 'PNW81879',
 'PNW85327',
 'PNW81129',
 'PNW86365',
 'PNW87085',
 'PNW71082',
 'PNW82895',
 'PNW76686',
 'PNW88959',
 'PNW75895',
 'PNW81433',
 'PNW79756',
 'PNW84822',
 'PNW84312',
 'PNW70725',
 'PNW84455',
 'PNW83041',
 'PNW84216',
 'PNW74303',
 'PNW81207',
 'PNW72081',
 'PNW78289',
 'PNW72419',
 'PNW76724',
 'PNW83419',
 'PNW77335',
 'PNW78275',
 'PNW84418',
 'PNW75932',
 'PNW79457',
 'PNW71164',
 'PNW75534',
 'PNW84756',
 'PNW83114',
 'PNW85944',
 'PNW79051',
 'PNW76243',
 'PNW71242',
 'PNW81058',
 'PNW86793',
 'PNW78697',
 'PNW82078',
 'PNW85529',
 'PNW83147',
 'PNW83146',
 'PNW72803',
 'PNW79491',
 'PNW76718',
 'PNW85308',
 'PNW74548',
 'PNW84755',
 'PNW78067',
 'PNW71967',
 'PNW83331',
 'PNW84742',
 'PNW76245',
 'PNW71696',
 'PNW81113',
 'PNW75000',
 'PNW73146',
 'PNW70798',
 'PNW75452',
 'PNW81880',
 'PNW71684',
 'PNW71383',
 'PNW79307',
 'PNW75255',
 'PNW75083',
 'PNW75084',
 'PNW82746',
 'PNW75016',
 'PNW87742',
 'PNW71518',
 'PNW74712',
 'PNW73881',
 'PNW74445',
 'PNW85634',
 'PNW82954',
 'PNW87308',
 'PNW80276',
 'PNW82481',
 'PNW71098',
 'PNW82576',
 'PNW71129',
 'PNW70618',
 'PNW73618',
 'PNW71576',
 'PNW75678',
 'PNW75953',
 'PNW83656',
 'PNW87918',
 'PNW83269',
 'PNW84748',
 'PNW82035',
 'PNW75279',
 'PNW73748',
 'PNW80451',
 'PNW73905',
 'PNW75625',
 'PNW87444',
 'PNW75006',
 'PNW71613',
 'PNW88433',
 'PNW81132',
 'PNW86457',
 'PNW85918',
 'PNW85685',
 'PNW86105',
 'PNW87000',
 'PNW87407',
 'PNW81467',
 'PNW80665',
 'PNW84710',
 'PNW79349',
 'PNW77002',
 'PNW71267',
 'PNW82756',
 'PNW72388',
 'PNW74186',
 'PNW87359',
 'PNW84254',
 'PNW70571',
 'PNW82125',
 'PNW87794',
 'PNW72578',
 'PNW85326',
 'PNW86865',
 'PNW73743',
 'PNW87855',
 'PNW78716',
 'PNW82949',
 'PNW71746',
 'PNW86713',
 'PNW73388',
 'PNW70325',
 'PNW76395',
 'PNW85312',
 'PNW82270',
 'PNW71946',
 'PNW76606',
 'PNW75181',
 'PNW74072',
 'PNW71982',
 'PNW84400',
 'PNW82929',
 'PNW75831',
 'PNW72113',
 'PNW71600',
 'PNW70505',
 'PNW75852',
 'PNW76266',
 'PNW81482',
 'PNW88323',
 'PNW80990',
 'PNW87520',
 'PNW72223',
 'PNW75677',
 'PNW83033',
 'PNW76022',
 'PNW74054',
 'PNW81253',
 'PNW80102',
 'PNW85857',
 'PNW81733',
 'PNW71004',
 'PNW85612',
 'PNW85030',
 'PNW73808',
 'PNW75217',
 'PNW79519',
 'PNW77619',
 'PNW78838',
 'PNW88639',
 'PNW87402',
 'PNW81856',
 'PNW86112',
 'PNW82595',
 'PNW83203',
 'PNW81756',
 'PNW75252',
 'PNW77088',
 'PNW73436',
 'PNW86073',
 'PNW79482',
 'PNW75150',
 'PNW82523',
 'PNW77458',
 'PNW83585',
 'PNW79458',
 'PNW74993',
 'PNW73533',
 'PNW86801',
 'PNW70364',
 'PNW83211',
 'PNW82956',
 'PNW73265',
 'PNW71373',
 'PNW86297',
 'PNW84721',
 'PNW70331',
 'PNW85617',
 'PNW70484',
 'PNW87204',
 'PNW70282',
 'PNW79155',
 'PNW71727',
 'PNW75623',
 'PNW71707',
 'PNW70654',
 'PNW77986',
 'PNW87981',
 'PNW74288',
 'PNW81277',
 'PNW80351',
 'PNW88129',
 'PNW82839',
 'PNW71962',
 'PNW78501',
 'PNW75967',
 'PNW83122',
 'PNW74396',
 'PNW86833',
 'PNW73665',
 'PNW70385',
 'PNW81047',
 'PNW81444',
 'PNW80803',
 'PNW82306',
 'PNW85188',
 'PNW87355',
 'PNW81877',
 'PNW72322',
 'PNW72452',
 'PNW81070',
 'PNW87179',
 'PNW88231',
 'PNW69645',
 'PNW72158',
 'PNW73316',
 'PNW80933',
 'PNW80092',
 'PNW71503',
 'PNW77020',
 'PNW77956',
 'PNW85353',
 'PNW84167',
 'PNW75128',
 'PNW80534',
 'PNW77426',
 'PNW74655',
 'PNW80611',
 'PNW80899',
 'PNW88295',
 'PNW88778',
 'PNW88509',
 'PNW87289',
 'PNW85021',
 'PNW81498',
 'PNW80697',
 'PNW70783',
 'PNW82356',
 'PNW71571',
 'PNW85732',
 'PNW85176',
 'PNW69923',
 'PNW87290',
 'PNW75251',
 'PNW71610',
 'PNW87346',
 'PNW70922',
 'PNW72144',
 'PNW84963',
 'PNW71871',
 'PNW88483',
 'PNW81321',
 'PNW75914',
 'PNW80744',
 'PNW69799',
 'PNW69969',
 'PNW77037',
 'PNW83431',
 'PNW88147',
 'PNW71614',
 'PNW83116',
 'PNW78763',
 'PNW86747',
 'PNW75407',
 'PNW75119',
 'PNW78280',
 'PNW88051',
 'PNW74009',
 'PNW89068',
 'PNW88649',
 'PNW81023',
 'PNW83958',
 'PNW70722',
 'PNW88760',
 'PNW86895',
 'PNW88097',
 'PNW85597',
 'PNW86945',
 'PNW76759',
 'PNW80395',
 'PNW80895',
 'PNW84440',
 'PNW85536',
 'PNW88194',
 'PNW76737',
 'PNW72758',
 'PNW87913',
 'PNW79391',
 'PNW75627',
 'PNW87798',
 'PNW72045',
 'PNW87278',
 'PNW83287',
 'PNW76677',
 'PNW80669',
 'PNW73505',
 'PNW74706',
 'PNW86221',
 'PNW86495',
 'PNW88355',
 'PNW69695',
 'PNW75292',
 'PNW80517',
 'PNW85287',
 'PNW83289',
 'PNW71548',
 'PNW74943',
 'PNW86094',
 'PNW88432',
 'PNW70118',
 'PNW85460',
 'PNW77282',
 'PNW87629',
 'PNW71832',
 'PNW86706',
 'PNW86562',
 'PNW72975',
 'PNW79169',
 'PNW81599',
 'PNW81362',
 'PNW85161',
 'PNW85775',
 'PNW74528',
 'PNW79844',
 'PNW78768',
 'PNW74830',
 'PNW85103',
 'PNW77501',
 'PNW75527',
 'PNW83275',
 'PNW87868',
 'PNW88682',
 'PNW88143',
 'PNW70573',
 'PNW76720',
 'PNW79459',
 'PNW81199',
 'PNW82880',
 'PNW71352',
 'PNW79293',
 'PNW85257',
 'PNW84311',
 'PNW75473',
 'PNW87557',
 'PNW77134',
 'PNW70475',
 'PNW70138',
 'PNW75701',
 'PNW74856',
 'PNW83548',
 'PNW85298',
 'PNW77011',
 'PNW85476',
 'PNW78569',
 'PNW71685',
 'PNW81286',
 'PNW88476',
 'PNW79261',
 'PNW87947',
 'PNW74775',
 'PNW77702',
 'PNW82497',
 'PNW74462',
 'PNW78989',
 'PNW72021',
 'PNW84273',
 'PNW88420',
 'PNW75737',
 'PNW84908',
 'PNW70792',
 'PNW71636',
 'PNW82913',
 'PNW87399',
 'PNW72284',
 'PNW71589',
 'PNW88239',
 'PNW72830',
 'PNW81169',
 'PNW87831',
 'PNW80152',
 'PNW71588',
 'PNW71013',
 'PNW74094',
 'PNW71593',
 'PNW80151',
 'PNW74574',
 'PNW80486',
 'PNW80482',
 'PNW77994',
 'PNW77995',
 'PNW70004',
 'PNW70003',
 'PNW72612',
 'PNW72613',
 'PNW82516',
 'PNW74572',
 'PNW77410',
 'PNW82515',
 'PNW70712',
 'PNW83518',
 'PNW83519',
 'PNW73603',
 'PNW73604',
 'PNW77409',
 'PNW70714',
 'PNW80286',
 'PNW85371',
 'PNW82150',
 'PNW82151',
 'PNW82154',
 'PNW82157',
 'PNW82963',
 'PNW82146',
 'PNW82147',
 'PNW85372',
 'PNW84235',
 'PNW79238',
 'PNW76324',
 'PNW79239',
 'PNW88088',
 'PNW88089',
 'PNW80287',
 'PNW76323',
 'PNW80169',
 'PNW69888',
 'PNW81094',
 'PNW73217',
 'PNW85958',
 'PNW86296',
 'PNW84813',
 'PNW84811',
 'PNW84812',
 'PNW87636',
 'PNW75253',
 'PNW87637',
 'PNW87635',
 'PNW87634',
 'PNW75636',
 'PNW78123',
 'PNW74034',
 'PNW82962',
 'PNW88180',
 'PNW84237',
 'PNW84236',
 'PNW85697',
 'PNW73187',
 'PNW73188',
 'PNW74283',
 'PNW84198',
 'PNW79159',
 'PNW70382',
 'PNW70012',
 'PNW75812',
 'PNW83990',
 'PNW79160',
 'PNW84143',
 'PNW80320',
 'PNW76752',
 'PNW84605',
 'PNW73061',
 'PNW88178']

In [ ]:
polya=polya_in_nanopolish('tests/polya.txt')
polya.head()

,readname,contig,position,leader_start,adapter_start,polya_start,transcript_start,read_rate,polya_length,qc_tag
0,6173a5c4-2dd2-4566-896d-2e7c34759c9e,1,1368,2.0,3.0,4451.0,5078.0,125.50,21.08,PASS
1,c029788b-c3bd-48f6-9f80-a6f361e381cb,1,16573,2.0,3.0,2911.0,3661.0,143.43,30.67,PASS
2,d1961345-ee5d-40ec-ba55-379e5daa5072,1,16573,110.0,279.0,4413.0,5814.0,125.50,53.33,PASS
3,52039941-5c98-49ef-971d-4437890e1b95,1,16573,2.0,3.0,4618.0,5769.0,130.96,45.00,PASS
4,b99e4287-065a-420a-b2ed-8a245039bb87,1,16573,2.0,3.0,3434.0,4164.0,125.50,25.37,PASS


In [ ]:
polya_length_correlation(bam,polya_nanopolish,tpm,'tests/',contigs=gene_list)

Cor:-0.2521544667116018,and P value: 8.355858576156513e-10
